# ag-viz Visualization Demo

This notebook demonstrates the visualization capabilities of the `ag-viz` package for ARIMA-GARCH models.

## Setup

First, ensure you have:
1. Built the C++ `ag` CLI tool
2. Installed the `ag-viz` package: `pip install -e python/`

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from ag_viz import (
    load_csv_data,
    load_model_json,
    plot_fit_diagnostics,
    plot_forecast,
    plot_residual_diagnostics,
    plot_simulation_paths,
    run_ag_command,
    find_ag_executable
)

# Check if ag executable is available
ag_path = find_ag_executable()
if ag_path:
    print(f"✓ Found ag executable at: {ag_path}")
else:
    print("⚠ Warning: ag executable not found. Set AG_EXECUTABLE environment variable.")

## 1. Fit a Model

First, let's fit an ARIMA-GARCH model to some example data.

In [ ]:
# Path to example data
data_path = Path('../examples/returns.csv')
model_path = Path('demo_model.json')

# Load the data to visualize
data = load_csv_data(data_path)
print(f"Loaded {len(data)} observations")
print(data.head())

# Plot the raw data
plt.figure(figsize=(12, 4))
plt.plot(data.iloc[:, 0].values)
plt.title('Raw Time Series Data')
plt.xlabel('Observation')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Fit an ARIMA(1,0,1)-GARCH(1,1) model
result = run_ag_command([
    'fit',
    '-d', str(data_path),
    '-a', '1,0,1',
    '-g', '1,1',
    '-o', str(model_path)
])

print(result.stdout)

## 2. Generate Fit Diagnostics

Visualize the fitted model with summary statistics.

In [ ]:
# Load the fitted model
model = load_model_json(model_path)

# Generate fit diagnostics plot
output_dir = Path('./demo_output')
fit_plot = plot_fit_diagnostics(data, model, output_dir, show=True)
print(f"✓ Saved fit diagnostics to: {fit_plot}")

## 3. Generate Forecasts

Create forecasts with confidence intervals.

In [ ]:
# Generate 30-step ahead forecast
forecast_path = Path('demo_forecast.csv')

result = run_ag_command([
    'forecast',
    '-m', str(model_path),
    '-n', '30',
    '-o', str(forecast_path)
])

print(result.stdout)

# Visualize the forecast
forecast_plot = plot_forecast(
    model_path,
    forecast_path,
    confidence_levels=[0.68, 0.95],
    show=True,
    save=output_dir / 'forecast.png'
)

print(f"✓ Saved forecast plot to: {forecast_plot}")

## 4. Residual Diagnostics

Generate comprehensive residual diagnostic plots.

In [ ]:
# Run diagnostics
diag_dir = output_dir / 'diagnostics'
diag_json = diag_dir / 'diagnostics.json'

result = run_ag_command([
    'diagnostics',
    '-m', str(model_path),
    '-d', str(data_path),
    '-o', str(diag_json)
])

print(result.stdout)

# Generate residual diagnostic plots
residual_plot = plot_residual_diagnostics(
    model_path,
    data_path,
    diag_json if diag_json.exists() else None,
    diag_dir
)

print(f"✓ Saved residual diagnostics to: {residual_plot}")

# Display the plot
from IPython.display import Image, display
display(Image(filename=str(residual_plot)))

## 5. Simulate Multiple Paths

Generate and visualize simulation paths from the fitted model.

In [ ]:
# Simulate 50 paths with 200 observations each
simulation_path = Path('demo_simulation.csv')

result = run_ag_command([
    'simulate',
    '-m', str(model_path),
    '-p', '50',
    '-n', '200',
    '-s', '42',
    '-o', str(simulation_path),
    '--stats'
])

print(result.stdout)

# Visualize simulation paths
sim_plot = plot_simulation_paths(
    simulation_path,
    n_paths_to_plot=15,
    output_path=output_dir / 'simulation.png',
    show=True
)

print(f"✓ Saved simulation plot to: {sim_plot}")

## 6. Customizing Plots

You can customize plots programmatically using matplotlib.

In [ ]:
# Example: Customize plot style
import matplotlib.pyplot as plt
import seaborn as sns

# Set a different style
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Generate a plot with custom style
custom_plot = plot_forecast(
    model_path,
    forecast_path,
    confidence_levels=[0.50, 0.90, 0.99],
    show=True,
    save=output_dir / 'custom_forecast.png'
)

print(f"✓ Saved custom forecast plot to: {custom_plot}")

# Reset to default style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## Summary

This demo showed:
1. Fitting an ARIMA-GARCH model to data
2. Generating fit diagnostic plots
3. Creating forecast visualizations with confidence intervals
4. Producing comprehensive residual diagnostics
5. Visualizing multiple simulation paths
6. Customizing plot styles

All visualizations are saved to the `demo_output` directory for further use.

## Next Steps

- Try different ARIMA-GARCH model specifications
- Experiment with different confidence levels
- Adjust the number of simulation paths
- Customize plot styles for publication
- Integrate with your own time series data